In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

In [21]:
import torch
import torchvision
print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')

torch version: 2.1.1
torchvision version: 0.16.1


In [22]:
# !pip install transformers
# !pip install datasets
# !pip install evaluate

In [23]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [24]:
# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

사용 디바이스: cuda:0


In [25]:
# !pwd

In [26]:
import pandas as pd
train = pd.read_csv('./patent300.csv')
test = pd.read_csv('./patent_valid.csv')
print('train',train.shape)
print('test',test.shape)


train (169200, 5)
test (56400, 5)


In [27]:
print(train.iloc[0])

Unnamed: 0                            0
id                        1019790000595
title         사료용 청목초(靑牧草)의 재배방법 및 재배장치
label                                 0
class                    A_01_011_01110
Name: 0, dtype: object


In [28]:
MODEL_NAME = 'kykim/bert-kor-base'
# MODEL_NAME='kykim/albert-kor-base'

In [29]:
import torch
from transformers import BertTokenizerFast, AlbertModel
from torch.utils.data import Dataset, DataLoader


class PatentDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe
        # Huggingface 토크나이저 생성

        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx): # 클래스의 인덱스에 접근할 때 자동으로 호출되는 메서드
        # iloc : 행번호로 선택하는 방법
        sentence = self.data.iloc[idx]['title']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)       # 2D -> 1D : 1,512 -> 512
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

/home/ubuntu/anaconda3/envs/aip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# 토크나이저 지정
tokenizer_pretrained = MODEL_NAME

# train, test 데이터셋 생성
train_data = PatentDataset(train, tokenizer_pretrained)
test_data = PatentDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)
# train_loader과 test_loader는 iterator이다.
# iterator란 반복을 이용해 어떤 처리를 수행하는 객체를 말한다.
print('train_loader',train_loader)
print('test_loader',test_loader)

tokenizer_config.json: 100%|██████████| 80.0/80.0 [00:00<00:00, 47.2kB/s]
vocab.txt: 100%|██████████| 344k/344k [00:00<00:00, 1.72MB/s]
config.json: 100%|██████████| 725/725 [00:00<00:00, 404kB/s]


train_loader <torch.utils.data.dataloader.DataLoader object at 0x7f408266db80>
test_loader <torch.utils.data.dataloader.DataLoader object at 0x7f406f7e3730>


In [31]:
# print(train_data[1])

In [32]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}

In [33]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

In [34]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [35]:
# label 출력
print(labels)

tensor([533, 540, 123, 496, 401, 327, 500, 170])


In [36]:
from transformers import BertModel
from transformers import AlbertModel

# 모델 생성
model = BertModel.from_pretrained(MODEL_NAME).to(device)
model

pytorch_model.bin: 100%|██████████| 476M/476M [00:02<00:00, 185MB/s]  


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [37]:
output = model(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [38]:
print(output['last_hidden_state'].shape)
print(output['pooler_output'].shape)

torch.Size([8, 512, 768])
torch.Size([8, 768])


In [39]:
print(output['last_hidden_state'][:,0,:])

tensor([[ 0.3693, -0.2431,  0.9074,  ...,  0.0818, -0.0605,  0.1202],
        [ 0.3885, -0.1453,  0.6115,  ..., -0.1577, -0.5622,  0.2863],
        [-0.7106, -0.5234, -0.6464,  ..., -0.1366,  0.0813,  0.1761],
        ...,
        [-0.1014, -0.4453,  0.6239,  ..., -0.5866,  0.3212,  0.3021],
        [ 0.6125,  0.4511, -0.4804,  ..., -0.8695, -1.1409,  0.5427],
        [ 0.0524,  0.2272, -0.2401,  ..., -0.1747, -0.6495,  1.0971]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [40]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(output['last_hidden_state'][:, 0, :])
print(fc_output.shape)
print(fc_output)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([[-0.8910,  0.4180],
        [-0.8913,  0.5254],
        [ 0.4962,  0.3876],
        [ 0.6888,  0.4491],
        [-0.0404,  0.3802],
        [-0.9704,  0.5465],
        [ 0.6677,  0.5718],
        [-0.9306,  0.5796]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([1, 1, 0, 0, 1, 1, 0, 1], device='cuda:0')


In [41]:
class FineTuningBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super().__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)

        # dropout 설정
        self.dropout = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 564)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dropout(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [42]:
BERT_model = FineTuningBertModel(MODEL_NAME)
BERT_model.to(device)

FineTuningBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [43]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(BERT_model.parameters(), lr=1e-5)

In [44]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()

        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)

        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)

        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()

        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()

        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        # print('output',output.shape) # (8, 2)
        _, index = output.max(dim=1)

        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += index.eq(labels).sum().item()
        counts += len(labels)

        # loss 값은 1개 배치의 평균 손실(loss) 입니다. labels.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 labels.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)

        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        # 배치당 training loss,

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [45]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()

    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0

        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs) # (8,2)

            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, index = output.max(dim=1)

            # idx.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(index.eq(labels)).item()

            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)

        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [46]:
# 최대 Epoch을 지정합니다.
num_epochs = 3

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(BERT_model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(BERT_model, test_loader, loss_fn, device)

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(BERT_model.state_dict(), f'{model_name}.pth')

    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 26.91349, training accuracy: 0.34050: 100%|██████████| 21150/21150 [1:26:11<00:00,  4.09batch/s]


[INFO] val_loss has been improved from inf to 2.34846. Saving Model!
epoch 01, loss: 3.36419, acc: 0.34050, val_loss: 2.34846, val_accuracy: 0.44266


training loss: 14.09020, training accuracy: 0.55150: 100%|██████████| 21150/21150 [1:26:12<00:00,  4.09batch/s]


[INFO] val_loss has been improved from 2.34846 to 2.17095. Saving Model!
epoch 02, loss: 1.76127, acc: 0.55150, val_loss: 2.17095, val_accuracy: 0.47293


training loss: 11.53720, training accuracy: 0.60865: 100%|██████████| 21150/21150 [1:26:11<00:00,  4.09batch/s]


epoch 03, loss: 1.44215, acc: 0.60865, val_loss: 2.18090, val_accuracy: 0.47447


In [66]:
torch.save(BERT_model.state_dict(), 'model.pth')